In [1]:
%%spark

import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, minute, second, dayofweek

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1623009074507_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
#input_data = "s3a://udacity-dend/"
#songs_table_raw = spark.read.json(input_data+"song_data/A/A/A/*.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
songs_table_raw = spark.read.json("s3a://udacity-dend/song_data/A/A/A/*.json")
#songs_table_raw = spark.read.json("s3a://udacity-dend/song_data/A/A/A/*.json")

df = songs_table_raw  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
songs_table = df.select("song_id","title","artist_id","year","duration")
 #val df1 = sourceDF.select("first column", "second column", "third column")
songs_table.limit(1).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(song_id='SOAFBCP12A8C13CC7D', title='King Of Scurf (2007 Digital Remaster)', artist_id='ARTC1LV1187B9A4858', year=1972, duration=301.40036)]

In [4]:
songs_table.write.partitionBy("year","artist_id").mode('overwrite').parquet("s3://aws-emr-resources-528452576569-us-east-2/songs_table.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
artists_table = df.select(col("artist_id")\
                          ,col("artist_name").alias("name")\
                          ,col("artist_location").alias("location")\
                          ,col("artist_latitude").alias("latitude")\
                          ,col("artist_longitude").alias("longitude") )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
artists_table.write.mode('overwrite').parquet("s3a://aws-emr-resources-528452576569-us-east-2/artists_table.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
songs_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: long (nullable = true)
 |-- duration: double (nullable = true)

In [8]:
artists_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)

## Log Data


In [9]:
df = spark.read.json("s3a://udacity-dend/log_data/2018/11/*.json")

#medium_load = df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df = df.filter(df.page == 'NextSong')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from pyspark.sql.types import TimestampType
def get_timestamp_fn(time_column):
  return datetime.fromtimestamp(time_column/1000)


get_timestamp = udf(get_timestamp_fn, TimestampType())

#datetime_time = datetime.datetime.fromtimestamp(1541207073796/1000)
#type(datetime_time)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df = df.withColumn("date_time_converted", get_timestamp("ts"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df = df.withColumn("hour", hour(col("date_time_converted")))\
            .withColumn("day", dayofmonth(col("date_time_converted")))\
            .withColumn("week", weekofyear(col("date_time_converted")))\
            .withColumn("month", month(col("date_time_converted")))\
            .withColumn("year", year(col("date_time_converted")))\
            .withColumn("weekday", dayofweek(col("date_time_converted")))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- date_time_converted: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)

In [15]:
df.select("date_time_converted","hour","day","week","month","year","weekday").show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---+----+-----+----+-------+
| date_time_converted|hour|day|week|month|year|weekday|
+--------------------+----+---+----+-----+----+-------+
|2018-11-15 00:30:...|   0| 15|  46|   11|2018|      5|
+--------------------+----+---+----+-----+----+-------+
only showing top 1 row

In [16]:
# extract columns to create time table
time_table = df.select(col("date_time_converted").alias("start_time")\
                          ,col("hour")\
                          ,col("day")\
                          ,col("week")\
                          ,col("month") \
                          ,col("year")\
                          ,col("weekday"))
# write time table to parquet files partitioned by year and month
time_table.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---+----+-----+----+-------+
|          start_time|hour|day|week|month|year|weekday|
+--------------------+----+---+----+-----+----+-------+
|2018-11-15 00:30:...|   0| 15|  46|   11|2018|      5|
+--------------------+----+---+----+-----+----+-------+
only showing top 1 row

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 56108)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulator

In [17]:
time_table.write.partitionBy("year","month").mode('overwrite').parquet("s3://aws-emr-resources-528452576569-us-east-2/timetable.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
time_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)

In [18]:
users_table = df.select(col("userId").alias("user_id")\
                          ,col("firstName").alias("first_name")\
                          ,col("lastName").alias("last_name")\
                          ,col("gender")\
                          ,col("level"))

# write users table to parquet files
#artists_table

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
users_table.write.mode('overwrite').parquet("s3://aws-emr-resources-528452576569-us-east-2/users_table.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
users_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

In [22]:
songs_table_raw.show(1)
df.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+--------------------+----------------+------------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|       artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+------------------+---------+---------+------------------+--------------------+----+
|ARTC1LV1187B9A4858|        51.4536|Goldsmith's Colle...|        -0.01802|The Bonzo Dog Band|301.40036|        1|SOAFBCP12A8C13CC7D|King Of Scurf (20...|1972|
+------------------+---------------+--------------------+----------------+------------------+---------+---------+------------------+--------------------+----+
only showing top 1 row

+--------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+-------------+------+-------------+-----

In [26]:
songplays_table = df.join(songs_table_raw.select("artist_name","duration","title","song_id","artist_id"), (df.artist == songs_table_raw.artist_name) 
                          & (df.length == songs_table_raw.duration) 
                          & (df.song == songs_table_raw.title))
songplays_table.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+-------------------+----+---+----+-----+----+-------+-----------+--------+-----+-------+---------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId|song|status| ts|userAgent|userId|date_time_converted|hour|day|week|month|year|weekday|artist_name|duration|title|song_id|artist_id|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+-------------------+----+---+----+-----+----+-------+-----------+--------+-----+-------+---------+
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+-------------------+----+---+----+-----+----+-------+-----------+--------+-----+-------+---------+

In [27]:
songplays_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- date_time_converted: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)
 |-- artist_name: string 

In [28]:
songplays_table = songplays_table.select(col("sessionId").alias("songplay_id")\
                          ,col("date_time_converted").alias("start_time")\
                          ,col("userId").alias("user_id")\
                          ,col("level")\
                          ,col("song_id")\
                          ,col("artist_id")\
                          ,col("sessionId").alias("session_id")\
                          ,col("location")\
                          ,col("userAgent").alias("user_agent")\
                          ,col("year")\
                          ,col("month"))

# write users table to parquet files
#artists_table

# write users table to parquet files
#artists_table

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…